- Create a RAG system for Satoshi Nakamoto's writings.
- Use DeepEval to create synthetic test dataset with questions.
- Calculate metrics to evaluate variations of the RAG system.

In [1]:
# IMPORTANT: Patch OpenAI for tracing BEFORE any deepeval imports
import openai
from deepeval.openai.patch import patch_openai
patch_openai(openai)

In [2]:
import os
import json
import glob
from dotenv import load_dotenv
from deepeval.tracing import observe
load_dotenv()
# Disable Chroma telemetry to avoid PostHog capture warnings
os.environ.setdefault('ANONYMIZED_TELEMETRY', 'False')

# Disable Chroma telemetry to suppress warnings
os.environ["ANONYMIZED_TELEMETRY"] = "False"

# Disable telemetry to suppress warnings
os.environ['DEEPEVAL_TELEMETRY_OPT_OUT'] = 'true'
os.environ['ANONYMIZED_TELEMETRY'] = 'false'  # For ChromaDB

# Ensure no leftover Deepeval Ollama defaults interfere
os.environ.pop('DEEPEVAL_EMBEDDINGS_PROVIDER', None)
os.environ.pop('DEEPEVAL_OLLAMA_BASE_URL', None)

## Collect all documents from scraping

In [3]:
# Step 1: Function to collect all .txt paths and map to URLs from source_map.json
def collect_document_paths_and_sources(main_folder: str):
    document_paths = []
    path_to_url = {}
    
    # Walk through all subfolders
    for root, dirs, files in os.walk(main_folder):
        # Check for source_map.json in this subfolder
        source_map_path = os.path.join(root, 'source_map.json')
        source_map = {}
        if os.path.exists(source_map_path):
            with open(source_map_path, 'r') as f:
                source_map = json.load(f)
        
        # Find all .txt files in this subfolder
        txt_files = glob.glob(os.path.join(root, '*.txt'))
        for txt_path in txt_files:
            basename = os.path.basename(txt_path)
            url = source_map.get(basename, 'unknown')  # Get URL if mapped, else 'unknown'
            document_paths.append(txt_path)
            path_to_url[txt_path] = url
    
    return document_paths, path_to_url

# main_folder = r'C:\GitRepos\LangChainCourse\documentation_assistant\raw_documents' 
main_folder = r'C:\GitRepos\RAG_learning\DeepEvalTests\raw_documents' 
document_paths, path_to_url = collect_document_paths_and_sources(main_folder)
print(f"Found {len(document_paths)} .txt files.")

Found 617 .txt files.


In [4]:
# Alternatively, only use the document paths after filtering with filter_documents.py (removing very short entries and duplicates)

with open('filtered_document_paths.json', 'r') as f:
    filtered_data = json.load(f)
document_paths = filtered_data['filtered_document_paths']

print(f"Using {len(document_paths)} .txt files.")

Using 545 .txt files.


### OBSOLETE:  Bring documents into correct format


DeepEval Synthesizer supports two formats:
- generate_goldens_from_docs: List of file paths to pdf txt files.
- generate_goldens_from_contexts: what we will use. List of lists of strings [["chunk1", "chunk2"], ["chunk3"]] -> thus we can use

Since Synthesizer just uses a list of list of the raw documents, we lose other metadata like the source of the document. Thus, below we create in parallel list of contexts and list of sources with same indexing.
Later we can attach that metadata again to the DeepEval Goldens which support additional fields.

In [5]:
# from typing import List, Dict


# # Step 1: Prepare contexts and parallel sources from your DataFrame
# contexts: List[List[str]] = []
# sources: List[str] = []
# unique_sequences = original_df['sequence_number'].unique()
# for seq_num in sorted(unique_sequences):  # Sort for consistent order
#     group = original_df[original_df['sequence_number'] == seq_num].sort_values('chunk_index')
#     # Extract content as list of strings (handle NaN if any)
#     chunk_contents = group['content'].astype(str).dropna().tolist()
#     if chunk_contents:  # Skip empty documents
#         # Get the source (assume unique per group; take first if multiple)
#         source = group['source'].dropna().unique()[0] if 'source' in group.columns and not group['source'].dropna().empty else 'unknown'
#         contexts.append(chunk_contents)
#         sources.append(source)

## Generate Synthetic Test Dataset

In [6]:
LLM_MODEL = "gpt-4o-mini"
EMBEDDING_MODEL = "text-embedding-3-small"

In [7]:
from deepeval.models import GPTModel, OpenAIEmbeddingModel

# Step 2: Set up OpenAI models with proper timeout and retry settings
# Requires env vars: OPENAI_API_KEY (and OPENAI_API_BASE if using Azure/other base)
llm = GPTModel(
    model=LLM_MODEL
)
embedding_model = OpenAIEmbeddingModel(
    model=EMBEDDING_MODEL
)

# Quick sanity check to ensure LLM calls succeed before long synth runs
# try:
#     _sanity_output, _ = llm.generate("Say 'ok'.")
#     print("LLM sanity check:", _sanity_output[:40])
# except Exception as e:
#     print("LLM sanity check failed:", repr(e))


In [8]:
from deepeval.synthesizer import Synthesizer
from deepeval.synthesizer.config import EvolutionConfig, ContextConstructionConfig, FiltrationConfig

evolution_config = EvolutionConfig(num_evolutions=0)

context_cfg = ContextConstructionConfig(
    critic_model=llm,
    embedder=embedding_model,
    max_contexts_per_document=2,  # Reduced from 3 to 2 to lower load
    context_quality_threshold=0.6,  # Slightly lowered threshold
    context_similarity_threshold=0.3, 
)

filtration_cfg = FiltrationConfig(
    critic_model=llm,
    synthetic_input_quality_threshold=0.5,
    max_quality_retries=1,  # Reduced retries to avoid timeout loops
)

synthesizer = Synthesizer(
    model=llm,
    async_mode=True,  # Use sync mode to avoid hanging issues
    evolution_config=evolution_config,
    filtration_config=filtration_cfg,
    max_concurrent=16  # Reduced to 1 to avoid overwhelming the API
)

In [9]:
# lets use a smaller subset for testing to avoid timeouts
# selected_document_paths = document_paths[113:120]
selected_document_paths = document_paths  
print(f"Processing {len(selected_document_paths)} documents for testing")


Processing 545 documents for testing


In [10]:
# # Generation test without custom processing
# raw_goldens = synthesizer.generate_goldens_from_docs(
#     document_paths=[selected_document_paths[0]],
#     max_goldens_per_context=1,
#     include_expected_output=True,
#     context_construction_config=context_cfg
# )

# # Check the first golden's metadata
# print(raw_goldens[0])

In [11]:
# Step 4: Generate goldens with clean error handling
import time
import shutil
from typing import List

def cleanup_vector_db():
    """Clean up any corrupted vector database files"""
    db_paths = ['.vector_db', '.chroma_db', 'chroma.sqlite3']
    for db_path in db_paths:
        if os.path.exists(db_path):
            try:
                if os.path.isdir(db_path):
                    shutil.rmtree(db_path)
                else:
                    os.remove(db_path)
                print(f"Cleaned up {db_path}")
            except Exception as e:
                print(f"Could not clean up {db_path}: {e}")

def process_and_save_goldens(goldens, path_to_url):
    """Process goldens and save to file"""
    for golden in goldens:
        # Use the source_file that DeepEval already set
        source_path = golden.source_file
        if source_path:
            url = path_to_url.get(source_path, 'unknown')
            if not hasattr(golden, 'additional_metadata') or golden.additional_metadata is None:
                golden.additional_metadata = {}
            golden.additional_metadata['source_url'] = url
        else:
            print(f"Warning: No source_file found for a golden.")
    
    print(f"Total generated {len(goldens)} goldens.")
    with open('synthetic_dataset_with_urls.json', 'w') as f:
        json.dump([g.__dict__ for g in goldens], f, indent=4)

@observe(name="Generate All Goldens")
def generate_all_goldens(synthesizer, paths, context_cfg):
    print(f"Starting generation for {len(paths)} documents...")
    start_time = time.time()
    
    try:
        goldens = synthesizer.generate_goldens_from_docs(
            document_paths=paths,
            max_goldens_per_context=2,
            include_expected_output=True,
            context_construction_config=context_cfg
        )
        
        elapsed = time.time() - start_time
        print(f"Generation completed in {elapsed:.2f} seconds")
        return goldens
        
    except Exception as e:
        elapsed = time.time() - start_time
        print(f"Generation failed after {elapsed:.2f} seconds with error: {e}")
        raise


# Clean up database and generate goldens
# print("Cleaning up any corrupted vector database...")
# cleanup_vector_db()

# print("Starting golden generation...")

# all_goldens = generate_all_goldens(synthesizer, selected_document_paths, context_cfg)
# process_and_save_goldens(all_goldens, path_to_url)
    


In [12]:
# NEW ROBUST SYNTHESIS - Use this instead of the failing cell above
import asyncio
from robust_synthesizer import run_robust_synthesis

# Use the full dataset instead of just 3 documents for production
# For testing, keep using selected_document_paths. For full processing, uncomment:
# selected_document_paths = document_paths  # Use all documents

print(f"Processing {len(selected_document_paths)} documents using robust synthesizer...")

# Run the robust synthesis with small batches and resume functionality
await run_robust_synthesis(
    document_paths=selected_document_paths,  # Change to document_paths for full dataset
    path_to_url=path_to_url,
    batch_size=3,  # Process 3 documents at a time (very conservative)
    max_concurrent=8  # Conservative concurrency
)

print("Robust synthesis completed! Check synthesis_progress.json for progress and synthetic_dataset_with_urls.json for results.")
print("If it fails partway through, you can simply re-run this cell to resume from where it left off.")


INFO:robust_synthesizer:Starting robust synthesis...


Processing 545 documents using robust synthesizer...


INFO:robust_synthesizer:Cleaned up .vector_db


[Confident AI Synthesizer Log] SUCCESS: Successfully deleted: .vector_db


INFO:robust_synthesizer:Loaded progress: 252/545 paths processed
INFO:robust_synthesizer:Resuming synthesis: 293 paths remaining out of 545
INFO:robust_synthesizer:Processing batch 4225: 3 paths
INFO:robust_synthesizer:Processing batch of 3 paths (attempt 1)


...

[Confident AI Synthesizer Log] SUCCESS: Successfully deleted: .vector_db

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


[Confident AI Synthesizer Log] SUCCESS: Successfully deleted: .vector_db

Adjust the `min_context_length` to no more than 0, or reduce `chunk_size`., retrying in 16s (attempt 5/6)


CancelledError: 